In [1]:
import os.path as osp

import torch

from torch_geometric.nn import SchNet, MessagePassing
from torch_geometric.data import DataLoader
from torch_geometric.datasets import QM9, Planetoid
import os
from pysmiles import read_smiles
from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
from tqdm import tqdm

from torch import nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader

In [2]:
!python -c "import torch; print(torch.__version__)"

1.7.1+cu110


In [3]:
!python -c "import torch; print(torch.version.cuda)"

11.0


In [4]:
train = pd.read_csv(os.path.join('..','data','train.csv'))
dev = pd.read_csv(os.path.join('..','data','dev.csv'))
train = pd.concat([train, dev])
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30345 entries, 0 to 70
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   uid            30345 non-null  object 
 1   SMILES         30345 non-null  object 
 2   S1_energy(eV)  30345 non-null  float64
 3   T1_energy(eV)  30345 non-null  float64
dtypes: float64(2), object(2)
memory usage: 1.2+ MB


In [5]:
train['target'] = train['S1_energy(eV)'] - train['T1_energy(eV)']

In [6]:
train

,uid,SMILES,S1_energy(eV),T1_energy(eV),target
0,train_0,CCC1CCCCN1C(=O)C(C)OC(=O)c1c(C)oc(-n2cccc2)c1C#N,4.6747,3.3809,1.2938
1,train_1,COc1ccc(Oc2ccc(N3C(=S)NC(c4ccccn4)C3c3cc(C)n(-...,3.6617,3.4585,0.2032
2,train_2,CC(=O)Nc1ccc(C(=O)[C@H](C)Sc2nnc(C3CCCCC3)o2)cc1,3.6420,3.1787,0.4633
3,train_3,OC(CNC1CC1)CN1CCc2sccc2C1,4.8901,3.7847,1.1054
4,train_4,CCNC(CCCC(F)(F)F)C1(OCC)CCOCC1,6.4967,6.2724,0.2243
...,...,...,...,...,...
66,dev_66,N#Cc1cc(-c2ccc(N3c4ccccc4Oc4ccccc43)cc2)c(-c2c...,2.1939,2.1846,0.0093
67,dev_67,CC1(C)c2ccccc2N(c2ccc(-c3nc4ccc(N5c6ccccc6C(C)...,2.3537,2.3371,0.0166
68,dev_68,Cc1cc(-n2c3ccc(C(C)(C)C)cc3c3cc(C(C)(C)C)ccc32...,2.1364,2.1260,0.0104
69,dev_69,Cc1cc(-n2c3ccccc3c3ccccc32)cc(C)c1B1c2ccccc2B(...,2.2650,2.2511,0.0139


In [7]:
mol_list = [Chem.MolFromSmiles(smi) for smi in train['SMILES']]

In [8]:
y_list = [y for y in train['target']]
len(y_list)

30345

In [9]:
len(mol_list)

30345

In [10]:
for mol, y in zip(mol_list, y_list):
    print(y)
    break

1.2937999999999996


In [11]:
def get_atom_features(mol):
    atomic_number = []
    num_hs = []
    
    for atom in mol.GetAtoms():
        atomic_number.append(atom.GetAtomicNum())
        num_hs.append(atom.GetTotalNumHs(includeNeighbors=True))
        
    return torch.tensor([atomic_number, num_hs]).t()

def get_edge_index(mol):
    row, col = [], []
    
    for bond in mol.GetBonds():
        start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        row += [start, end]
        col += [end, start]
        
    return torch.tensor([row, col], dtype=torch.long)

from torch_geometric.data.dataloader import DataLoader
import torch_geometric.data


def prepare_dataloader(mol_list, y_list):
    data_list = []

    for mol, target in zip(mol_list, y_list):

        x = get_atom_features(mol)
        edge_index = get_edge_index(mol)
        y = target
        data = torch_geometric.data.data.Data(x=x, y=y, edge_index=edge_index)
        data_list.append(data)

    return DataLoader(data_list, batch_size=32, shuffle=True), data_list

In [39]:
xx = get_atom_features(mol_list[0])
xx

tensor([[6, 3],
        [6, 2],
        [6, 1],
        [6, 2],
        [6, 2],
        [6, 2],
        [6, 2],
        [7, 0],
        [6, 0],
        [8, 0],
        [6, 1],
        [6, 3],
        [8, 0],
        [6, 0],
        [8, 0],
        [6, 0],
        [6, 0],
        [6, 3],
        [8, 0],
        [6, 0],
        [7, 0],
        [6, 1],
        [6, 1],
        [6, 1],
        [6, 1],
        [6, 0],
        [6, 0],
        [7, 0]])

In [12]:
dloader, dlist = prepare_dataloader(mol_list, y_list)

In [31]:
dlist

[Data(edge_index=[2, 60], x=[28, 2], y=1.2937999999999996),
 Data(edge_index=[2, 94], x=[42, 2], y=0.20320000000000027),
 Data(edge_index=[2, 56], x=[26, 2], y=0.4632999999999998),
 Data(edge_index=[2, 38], x=[17, 2], y=1.1054000000000004),
 Data(edge_index=[2, 40], x=[20, 2], y=0.2242999999999995),
 Data(edge_index=[2, 26], x=[13, 2], y=0.4264000000000001),
 Data(edge_index=[2, 36], x=[18, 2], y=1.0014000000000003),
 Data(edge_index=[2, 44], x=[21, 2], y=1.0229000000000004),
 Data(edge_index=[2, 60], x=[28, 2], y=0.4744999999999999),
 Data(edge_index=[2, 46], x=[22, 2], y=1.2083000000000004),
 Data(edge_index=[2, 36], x=[17, 2], y=1.4551000000000003),
 Data(edge_index=[2, 26], x=[13, 2], y=0.5714999999999999),
 Data(edge_index=[2, 46], x=[22, 2], y=0.2536999999999998),
 Data(edge_index=[2, 42], x=[21, 2], y=0.4117999999999995),
 Data(edge_index=[2, 54], x=[26, 2], y=1.5152999999999999),
 Data(edge_index=[2, 60], x=[28, 2], y=0.8511999999999995),
 Data(edge_index=[2, 42], x=[20, 2], y=

In [38]:
for i, batch in enumerate(dloader):
    print(batch.x)    
    if i == 20:
        break
print(batch.x)

tensor([[6, 3],
        [6, 2],
        [6, 2],
        ...,
        [7, 0],
        [6, 2],
        [6, 2]])
tensor([[8, 0],
        [6, 0],
        [6, 2],
        ...,
        [6, 2],
        [6, 2],
        [6, 2]])
tensor([[ 6,  3],
        [ 6,  2],
        [ 6,  2],
        ...,
        [ 6,  1],
        [ 6,  0],
        [16,  0]])
tensor([[8, 0],
        [6, 0],
        [6, 2],
        ...,
        [6, 0],
        [8, 0],
        [8, 1]])
tensor([[6, 3],
        [6, 1],
        [6, 2],
        ...,
        [8, 0],
        [6, 1],
        [6, 1]])
tensor([[6, 3],
        [8, 0],
        [6, 0],
        ...,
        [8, 0],
        [6, 1],
        [6, 1]])
tensor([[ 7,  2],
        [ 7,  1],
        [ 6,  0],
        ...,
        [ 7,  1],
        [ 6,  0],
        [16,  0]])
tensor([[6, 3],
        [6, 1],
        [6, 3],
        ...,
        [6, 3],
        [6, 3],
        [6, 1]])
tensor([[6, 3],
        [6, 0],
        [6, 1],
        ...,
        [6, 1],
        [6, 1],
   

In [16]:
dlist[0].x

tensor([[6, 3],
        [6, 2],
        [6, 1],
        [6, 2],
        [6, 2],
        [6, 2],
        [6, 2],
        [7, 0],
        [6, 0],
        [8, 0],
        [6, 1],
        [6, 3],
        [8, 0],
        [6, 0],
        [8, 0],
        [6, 0],
        [6, 0],
        [6, 3],
        [8, 0],
        [6, 0],
        [7, 0],
        [6, 1],
        [6, 1],
        [6, 1],
        [6, 1],
        [6, 0],
        [6, 0],
        [7, 0]])

In [17]:
dlist[0].y

1.2937999999999996

In [42]:
len(dlist)

30345

In [18]:
dlist[0].edge_index.t()

tensor([[ 0,  1],
        [ 1,  0],
        [ 1,  2],
        [ 2,  1],
        [ 2,  3],
        [ 3,  2],
        [ 3,  4],
        [ 4,  3],
        [ 4,  5],
        [ 5,  4],
        [ 5,  6],
        [ 6,  5],
        [ 6,  7],
        [ 7,  6],
        [ 7,  8],
        [ 8,  7],
        [ 8,  9],
        [ 9,  8],
        [ 8, 10],
        [10,  8],
        [10, 11],
        [11, 10],
        [10, 12],
        [12, 10],
        [12, 13],
        [13, 12],
        [13, 14],
        [14, 13],
        [13, 15],
        [15, 13],
        [15, 16],
        [16, 15],
        [16, 17],
        [17, 16],
        [16, 18],
        [18, 16],
        [18, 19],
        [19, 18],
        [19, 20],
        [20, 19],
        [20, 21],
        [21, 20],
        [21, 22],
        [22, 21],
        [22, 23],
        [23, 22],
        [23, 24],
        [24, 23],
        [19, 25],
        [25, 19],
        [25, 26],
        [26, 25],
        [26, 27],
        [27, 26],
        [ 7,  2],
        [ 

In [40]:
import torch
from torch.nn import Linear
import torch.nn.functional as F 
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
embedding_size = 64

class GCN(torch.nn.Module):
    def __init__(self):
        # Init parent
        super(GCN, self).__init__()
        torch.manual_seed(42)

        # GCN layers
        self.initial_conv = GCNConv(2, embedding_size)
        self.conv1 = GCNConv(embedding_size, embedding_size)
        self.conv2 = GCNConv(embedding_size, embedding_size)
        self.conv3 = GCNConv(embedding_size, embedding_size)

        # Output layer
        self.out = Linear(embedding_size*2, 1)

    def forward(self, x, edge_index, batch_index):
        # First Conv layer
        hidden = self.initial_conv(x, edge_index)
        hidden = F.tanh(hidden)

        # Other Conv layers
        hidden = self.conv1(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv2(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv3(hidden, edge_index)
        hidden = F.tanh(hidden)
          
        # Global Pooling (stack different aggregations)
        hidden = torch.cat([gmp(hidden, batch_index), 
                            gap(hidden, batch_index)], dim=1)

        # Apply a final (linear) classifier.
        out = self.out(hidden)

        return out, hidden

model = GCN()
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GCN(
  (initial_conv): GCNConv(2, 64)
  (conv1): GCNConv(64, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
Number of parameters:  12801


In [ ]:
from torch_geometric.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

# Root mean squared error
loss_fn = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0007)  

# Use GPU for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Wrap data in a data loader
data = dlist
data_size = len(data)
NUM_GRAPHS_PER_BATCH = 64
loader = DataLoader(data[:int(data_size * 0.8)], 
                    batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
test_loader = DataLoader(data[int(data_size * 0.8):], 
                         batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)


def train(data):
    # Enumerate over the data
    for batch in loader:
        # Use GPU
        batch.to(device)  
        # Reset gradients
        optimizer.zero_grad() 
        # Passing the node features and the connection info
        pred, embedding = model(batch.x.float(), batch.edge_index, batch.batch) 
        # Calculating the loss and gradients
        loss = loss_fn(pred, batch.y)
        loss.backward()  
        # Update using the gradients
        optimizer.step()   
    return loss, embedding

print("Starting training...")
losses = []
for epoch in range(2000):
    loss, h = train(data)
    losses.append(loss)
    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Train Loss {loss}")

Starting training...
Epoch 0 | Train Loss 0.27015265822410583
Epoch 10 | Train Loss 0.32737502455711365
Epoch 20 | Train Loss 0.2162427455186844
Epoch 30 | Train Loss 0.3440459370613098
Epoch 40 | Train Loss 0.29982790350914
Epoch 50 | Train Loss 0.2987600564956665
Epoch 60 | Train Loss 0.2284633070230484
Epoch 70 | Train Loss 0.37255802750587463
Epoch 80 | Train Loss 0.30166998505592346
Epoch 90 | Train Loss 0.35933470726013184
